# Step 1: Import data

In [2]:
import pandas as pd
import numpy as np
from itertools import combinations 
import statsmodels.api as sm

desired_width = 320
pd.set_option('display.width', desired_width)
np.set_printoptions(linewidth=desired_width)
pd.set_option('display.max_columns', 100)

In [3]:
df = pd.read_csv("data.csv")
df['excess_ret_lag']= df['excess_ret'].shift(1)
df = df[df['date'] != '1994-01-31']
df['constant'] = 1

In [12]:
import warnings
warnings.filterwarnings("ignore")

def ols_params(data,xcols,ycol):
    results = sm.OLS(data[ycol], data[xcols]).fit()
    return results.params

def ols_pvalues(data,xcols,ycol):
    results = sm.OLS(data[ycol], data[xcols]).fit()
    return results.pvalues

column_names = ['lg_size_lag','lg_bm_lag','lg_leverage_lag','excess_ret_lag','roa_lag','acc_lag',
    'pcf_lag','sale_totdebt_lag','sp_lag','volatility_lag','beta_lag','constant','CAPEI_lag']

dataframe = pd.DataFrame(columns = column_names)

ycol = 'excess_ret'

# Step 2: Run cross-sectional regression of all Combinations of factors

In [21]:
%%time

#range(1-3) 3min
#range(3-6) 1h 51min 50s
#range(6-8)
#range(8-9) 55min 17s
#range(9-10) 42min 17s
#range(10-14) 22min 50s

for x in range(8,9): ### Returns regression combinations of 1st # of factors (Min:1) to 2nd # of factors (Max:14)
    comb= combinations(['lg_size_lag','lg_bm_lag','lg_leverage_lag','excess_ret_lag','roa_lag','acc_lag',
    'pcf_lag','sale_totdebt_lag','sp_lag','volatility_lag','beta_lag','constant','CAPEI_lag'],x)

    for i in list(comb):
        comb_lst = list(i)
        char_lst = list(i)
        res = df.groupby('date').apply(ols_params, ycol = ycol , xcols=char_lst)

        tstat = abs(res.mean() / (res.std()/np.sqrt(len(res))))
        tstat_df = tstat.to_frame()
        print(tstat_df)
    
        xcols_2 = []
        xcols_2 = comb_lst
        xcols_2.insert(0,'permno')
        res_left = pd.merge(df[['permno','date']], res, on=['date'], how='left')
        res_final = res_left[xcols_2]

        m=60

        if 'lg_size_lag'in comb_lst:
            res_final['lg_size_avg'] = res_final['lg_size_lag'].rolling(window=m).mean()
        if 'lg_bm_lag'in comb_lst:
            res_final['lg_bm_avg'] = res_final['lg_bm_lag'].rolling(window=m).mean()
        if 'lg_leverage_lag'in comb_lst:
            res_final['lg_leverage_avg'] = res_final['lg_leverage_lag'].rolling(window=m).mean()
        if 'CAPEI_lag'in comb_lst:
            res_final['CAPEI_lag_avg'] = res_final['CAPEI_lag'].rolling(window=m).mean()
        if 'excess_ret_lag'in comb_lst:
            res_final['excess_ret_avg'] = res_final['excess_ret_lag'].rolling(window=m).mean()
        if 'roa_lag'in comb_lst:
            res_final['roa_avg'] = res_final['roa_lag'].rolling(window=m).mean()
        if 'acc_lag'in comb_lst:
            res_final['acc_avg'] = res_final['acc_lag'].rolling(window=m).mean()
        if 'pcf_lag'in comb_lst:
            res_final['pcf_avg'] = res_final['pcf_lag'].rolling(window=m).mean()
        if 'sale_totdebt_lag'in comb_lst:
            res_final['sale_totdebt_avg'] = res_final['sale_totdebt_lag'].rolling(window=m).mean()
        if 'sp_lag'in comb_lst:
            res_final['sp_avg'] = res_final['sp_lag'].rolling(window=m).mean()
        if 'volatility_lag'in comb_lst:
            res_final['volatility_avg'] = res_final['volatility_lag'].rolling(window=m).mean()
        if 'beta_lag'in comb_lst:
            res_final['beta_avg'] = res_final['beta_lag'].rolling(window=m).mean()
        if 'constant'in comb_lst:
            res_final['constant_avg'] = res_final['constant'].rolling(window=m).mean()
        else:
            pass

        res_avg = res_final[[k for k in res_final.columns if k.endswith('avg') or k.startswith('permno')]]
        res_avg = res_avg.groupby('permno', group_keys=False).apply(lambda x:x.iloc[59:])
        res_avg=res_avg.drop(columns=['permno'])
        X = df[xcols_2]
        X = X.groupby('permno', group_keys=False).apply(lambda x:x.iloc[59:])
        X = X.drop(columns=['permno'])
        multiply = pd.DataFrame(X.values*res_avg.values, columns=X.columns, index=X.index)
        multiply['y_head'] = multiply.sum(axis=1)

        y_and_y_head = pd.concat([df[['excess_ret']][m-1:],multiply[['y_head']]], axis=1)
        y_and_y_head['const'] = 1
        reg1 = sm.OLS(endog=y_and_y_head['excess_ret'], exog=y_and_y_head[['y_head','const']], missing='drop')
        results = reg1.fit()

        y_head = results.params.y_head
        const = results.params.const
        percentage_accuracy = len(y_and_y_head.loc[(y_and_y_head.excess_ret*y_and_y_head.y_head > 0)])/len(y_and_y_head)
        avg_pnl = (y_and_y_head.loc[(y_and_y_head.excess_ret*y_and_y_head.y_head > 0)].excess_ret.sum()+\
                y_and_y_head.loc[(y_and_y_head.excess_ret*y_and_y_head.y_head < 0) &\
                (y_and_y_head.y_head>0)].excess_ret.sum())/len(y_and_y_head)

        for j in range(len(tstat_df)):
            lastrow = len(dataframe)
            if j!=0:
                lastrow = len(dataframe)-1
                dataframe.loc[lastrow,tstat_df.index.values[j]] = tstat_df.iloc[j,0]
                dataframe.loc[lastrow,'Y_hat'] = y_head
                dataframe.loc[lastrow,'Constant'] = const
                dataframe.loc[lastrow,'% Accuracy'] = percentage_accuracy
                dataframe.loc[lastrow,'Avg_PnL'] = avg_pnl
            else:
                dataframe.loc[lastrow,tstat_df.index.values[j]] = tstat_df.iloc[j,0]
                dataframe.loc[lastrow,'Y_hat'] = y_head
                dataframe.loc[lastrow,'Constant'] = const
                dataframe.loc[lastrow,'% Accuracy'] = percentage_accuracy
                dataframe.loc[lastrow,'Avg_PnL'] = avg_pnl

                         0
lg_size_lag       1.096470
lg_bm_lag         2.222433
lg_leverage_lag   3.230108
excess_ret_lag    3.514177
roa_lag           2.266895
acc_lag           1.976999
pcf_lag           0.723402
sale_totdebt_lag  0.465768
                        0
lg_size_lag      0.601205
lg_bm_lag        1.310178
lg_leverage_lag  3.121958
excess_ret_lag   3.833139
roa_lag          1.800819
acc_lag          1.992333
pcf_lag          0.526362
sp_lag           3.452425
                        0
lg_size_lag      0.756380
lg_bm_lag        1.565137
lg_leverage_lag  3.009580
excess_ret_lag   4.464421
roa_lag          1.805113
acc_lag          1.678051
pcf_lag          0.665304
volatility_lag   2.056289
                        0
lg_size_lag      0.323661
lg_bm_lag        1.783353
lg_leverage_lag  3.307613
excess_ret_lag   4.936897
roa_lag          2.070807
acc_lag          1.657187
pcf_lag          0.741507
beta_lag         1.489903
                        0
lg_size_lag      2.943812
lg_

                        0
lg_size_lag      2.913626
lg_bm_lag        0.842957
lg_leverage_lag  2.557570
excess_ret_lag   3.321169
roa_lag          1.163374
pcf_lag          1.400804
constant         3.187084
CAPEI_lag        0.049606
                         0
lg_size_lag       0.822805
lg_bm_lag         1.244031
lg_leverage_lag   2.708254
excess_ret_lag    4.954936
roa_lag           2.404140
sale_totdebt_lag  0.873473
sp_lag            2.700781
volatility_lag    1.447759
                         0
lg_size_lag       0.005892
lg_bm_lag         1.118690
lg_leverage_lag   2.673141
excess_ret_lag    5.376725
roa_lag           2.279585
sale_totdebt_lag  0.458092
sp_lag            3.259765
beta_lag          1.216234
                         0
lg_size_lag       2.494670
lg_bm_lag         0.855515
lg_leverage_lag   2.458237
excess_ret_lag    3.641380
roa_lag           1.677351
sale_totdebt_lag  0.660496
sp_lag            2.551321
constant          2.621095
                         0
lg_size_la

                        0
lg_size_lag      2.871107
lg_bm_lag        0.457914
lg_leverage_lag  2.048047
excess_ret_lag   3.264851
acc_lag          1.455700
pcf_lag          1.249541
constant         3.266754
CAPEI_lag        0.160728
                         0
lg_size_lag       0.089134
lg_bm_lag         0.356356
lg_leverage_lag   1.611643
excess_ret_lag    4.941512
acc_lag           1.476490
sale_totdebt_lag  0.566282
sp_lag            2.870756
volatility_lag    1.494666
                         0
lg_size_lag       1.002990
lg_bm_lag         0.271257
lg_leverage_lag   1.570960
excess_ret_lag    5.357234
acc_lag           1.732740
sale_totdebt_lag  0.019255
sp_lag            3.463986
beta_lag          1.119835
                         0
lg_size_lag       2.418242
lg_bm_lag         0.240182
lg_leverage_lag   1.738016
excess_ret_lag    3.573293
acc_lag           1.438771
sale_totdebt_lag  0.620141
sp_lag            2.539345
constant          2.769245
                         0
lg_size_la

                        0
lg_size_lag      1.246031
lg_bm_lag        0.174362
lg_leverage_lag  1.732061
excess_ret_lag   5.323216
pcf_lag          0.874537
sp_lag           3.759665
beta_lag         1.119496
CAPEI_lag        0.539990
                        0
lg_size_lag      2.316520
lg_bm_lag        0.102451
lg_leverage_lag  1.786725
excess_ret_lag   3.525273
pcf_lag          1.360213
sp_lag           2.820300
constant         2.743230
CAPEI_lag        0.067453
                        0
lg_size_lag      2.842159
lg_bm_lag        0.258811
lg_leverage_lag  1.822400
excess_ret_lag   4.744740
pcf_lag          1.559251
volatility_lag   1.155630
beta_lag         0.362257
constant         3.041753
                        0
lg_size_lag      0.127140
lg_bm_lag        0.692319
lg_leverage_lag  1.960903
excess_ret_lag   5.065169
pcf_lag          0.933564
volatility_lag   2.593777
beta_lag         0.073057
CAPEI_lag        0.290862
                        0
lg_size_lag      2.741343
lg_bm_lag   

                        0
lg_size_lag      3.280136
lg_bm_lag        0.338458
lg_leverage_lag  2.508238
roa_lag          0.662895
acc_lag          1.412718
pcf_lag          0.900567
constant         3.397534
CAPEI_lag        0.285480
                         0
lg_size_lag       1.069408
lg_bm_lag         0.817642
lg_leverage_lag   2.659637
roa_lag           2.095129
acc_lag           1.354970
sale_totdebt_lag  0.876783
sp_lag            2.578134
volatility_lag    1.490225
                         0
lg_size_lag       0.348164
lg_bm_lag         0.730869
lg_leverage_lag   2.641173
roa_lag           2.050049
acc_lag           1.436227
sale_totdebt_lag  0.519484
sp_lag            3.231861
beta_lag          1.289528
                         0
lg_size_lag       2.904967
lg_bm_lag         0.382306
lg_leverage_lag   2.400259
roa_lag           1.166359
acc_lag           1.403157
sale_totdebt_lag  0.712366
sp_lag            2.420658
constant          2.894977
                         0
lg_size_la

...

                         0
lg_bm_lag         0.279784
pcf_lag           1.239430
sale_totdebt_lag  0.466112
sp_lag            3.001978
volatility_lag    0.780759
beta_lag          0.683772
constant          1.514024
CAPEI_lag         0.123988
                         0
lg_leverage_lag   1.470633
excess_ret_lag    4.896482
roa_lag           1.200228
acc_lag           1.061703
pcf_lag           1.005010
sale_totdebt_lag  0.758587
sp_lag            3.012630
volatility_lag    1.513000
                         0
lg_leverage_lag   2.196605
excess_ret_lag    5.183979
roa_lag           1.898363
acc_lag           1.519848
pcf_lag           1.056664
sale_totdebt_lag  0.751252
sp_lag            3.444665
beta_lag          1.259498
                         0
lg_leverage_lag   0.997154
excess_ret_lag    3.763873
roa_lag           0.198764
acc_lag           1.073331
pcf_lag           1.584642
sale_totdebt_lag  0.017690
sp_lag            3.603919
constant          1.835692
                         0
l

                        0
lg_leverage_lag  0.947554
excess_ret_lag   4.936263
roa_lag          0.408063
acc_lag          1.166923
volatility_lag   2.083972
beta_lag         0.090840
constant         0.593050
CAPEI_lag        0.184638
                         0
lg_leverage_lag   1.508852
excess_ret_lag    5.293692
roa_lag           1.312416
pcf_lag           1.100827
sale_totdebt_lag  0.578564
sp_lag            3.350895
volatility_lag    1.380679
beta_lag          0.456512
                         0
lg_leverage_lag   1.000220
excess_ret_lag    4.796569
roa_lag           0.864100
pcf_lag           1.568738
sale_totdebt_lag  0.414772
sp_lag            3.342478
volatility_lag    1.153550
constant          0.467604
                         0
lg_leverage_lag   1.565058
excess_ret_lag    4.943040
roa_lag           1.565020
pcf_lag           1.086038
sale_totdebt_lag  0.699665
sp_lag            3.142244
volatility_lag    1.418920
CAPEI_lag         0.290100
                         0
lg_leverag

                        0
lg_leverage_lag  0.855702
excess_ret_lag   5.269062
acc_lag          1.173210
pcf_lag          1.503109
sp_lag           3.512144
beta_lag         1.063166
constant         1.680384
CAPEI_lag        0.691901
                        0
lg_leverage_lag  1.002401
excess_ret_lag   4.906065
acc_lag          1.120453
pcf_lag          1.767473
volatility_lag   2.140099
beta_lag         0.035551
constant         0.623653
CAPEI_lag        0.250681
                         0
lg_leverage_lag   0.942162
excess_ret_lag    5.197682
acc_lag           1.086073
sale_totdebt_lag  0.395799
sp_lag            3.268954
volatility_lag    1.312413
beta_lag          0.427959
constant          0.509248
                         0
lg_leverage_lag   1.599218
excess_ret_lag    5.269009
acc_lag           1.530172
sale_totdebt_lag  0.616347
sp_lag            3.281620
volatility_lag    1.607932
beta_lag          0.469632
CAPEI_lag         0.356324
                         0
lg_leverage_lag   0

                        0
lg_leverage_lag  1.239650
roa_lag          0.399018
acc_lag          0.835459
sp_lag           3.439395
volatility_lag   1.145051
beta_lag         0.630124
constant         0.258754
CAPEI_lag        0.337265
                         0
lg_leverage_lag   0.924758
roa_lag           0.755618
pcf_lag           1.499114
sale_totdebt_lag  0.266194
sp_lag            3.374817
volatility_lag    1.028603
beta_lag          0.590700
constant          0.485726
                         0
lg_leverage_lag   1.423628
roa_lag           1.399815
pcf_lag           1.066526
sale_totdebt_lag  0.501713
sp_lag            3.172943
volatility_lag    1.206326
beta_lag          0.622515
CAPEI_lag         0.412224
                         0
lg_leverage_lag   0.909309
roa_lag           0.911226
pcf_lag           1.360738
sale_totdebt_lag  0.325739
sp_lag            3.129307
volatility_lag    1.130694
constant          0.470728
CAPEI_lag         0.292786
                         0
lg_leverag

                         0
excess_ret_lag    4.948969
roa_lag           0.779515
acc_lag           1.163886
sale_totdebt_lag  0.998457
volatility_lag    1.857149
beta_lag          0.226524
constant          1.176477
CAPEI_lag         0.220863
                       0
excess_ret_lag  5.238800
roa_lag         0.326397
acc_lag         1.049672
sp_lag          3.413623
volatility_lag  1.067990
beta_lag        0.631280
constant        1.047229
CAPEI_lag       0.484797
                         0
excess_ret_lag    5.186071
roa_lag           0.044478
pcf_lag           1.612271
sale_totdebt_lag  0.307202
sp_lag            3.386277
volatility_lag    0.993095
beta_lag          0.610541
constant          1.179143
                         0
excess_ret_lag    5.272434
roa_lag           1.413373
pcf_lag           0.864019
sale_totdebt_lag  0.104829
sp_lag            3.427532
volatility_lag    1.763272
beta_lag          0.747074
CAPEI_lag         0.735238
                         0
excess_ret_lag    4

In [22]:
dataframe.head()

,lg_size_lag,lg_bm_lag,lg_leverage_lag,excess_ret_lag,roa_lag,acc_lag,pcf_lag,sale_totdebt_lag,sp_lag,volatility_lag,beta_lag,constant,CAPEI_lag,Y_hat,Constant,% Accuracy,Avg_PnL
0,3.56877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.528545,0.003656,0.445931,0.006516
1,NaN,2.82243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.356360,0.010081,0.412699,-0.001883
2,NaN,NaN,3.57866,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.477134,0.004761,0.445931,0.006516
3,NaN,NaN,NaN,0.555267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.906664,0.008619,0.414867,-0.006895
4,NaN,NaN,NaN,NaN,3.71251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.205631,0.007253,0.443685,0.005817


In [23]:
###change file name to match the range you are doing
dataframe.to_csv ('Part_A_results_8.csv', index = False, header=True) 